**TripAdvisor**

In [5]:
# TripAdvisor Reviews
import requests
from bs4 import BeautifulSoup
import time
import random
import re
import collections
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})

**Getting All Initial Review Pages from all wanted Tourist Destinations**

In [6]:
site_category_list = ['https://www.tripadvisor.com.ph/Attractions-g294206-Activities-c47-Kenya.html', # Sights and Landmarks
                    'https://www.tripadvisor.com.ph/Attractions-g294206-Activities-c57-Kenya.html', # Nature and Parks
                     'https://www.tripadvisor.com.ph/Attractions-g294206-Activities-c49-Kenya.html'] # Museums
skipped_urls = []
first_page_list = []

In [7]:
def getFirstPages(x):
    for i in x:
        page = requests.get(i)
        soup = BeautifulSoup(page.content, 'html.parser')
        sites = soup.find_all(class_ = 'attraction_element')
        for j in sites:
            if j.find(class_ = 'rs rating') != None:
                first_page_list.append('https://www.tripadvisor.com.ph/' + j.find(class_ = 'listing_title').a.get('href'))
        if len(sites) == 0 and i not in skipped_urls:
            skipped_urls.append(i)
        if len(sites) > 0 and i in skipped_urls:
            skipped_urls.remove(i)
        print('skipped: ', skipped_urls)
        time.sleep(random.uniform(5, 15))
    if len(skipped_urls) != 0:
        print('restarting function...')
        getFirstPages(skipped_urls)

In [8]:
getFirstPages(site_category_list)

skipped:  []
skipped:  []
skipped:  []


In [9]:
print(collections.Counter(first_page_list) == collections.Counter(list(set(first_page_list))))
print(len(first_page_list))

False
90


In [10]:
first_page_list

['https://www.tripadvisor.com.ph//Attraction_Review-g480204-d1643758-Reviews-Kiambethu_Farm-Limuru_Central_Province.html',
 'https://www.tripadvisor.com.ph//Attraction_Review-g1600222-d1924858-Reviews-Observation_Hill-Amboseli_National_Park_Amboseli_Eco_system_Rift_Valley_Province.html',
 'https://www.tripadvisor.com.ph//Attraction_Review-g294207-d1607150-Reviews-Karen_Blixen_Coffee_Garden_and_Cottages-Nairobi.html',
 'https://www.tripadvisor.com.ph//Attraction_Review-g661279-d1131514-Reviews-Bio_Ken_Snake_Farm-Watamu_Coast_Province.html',
 'https://www.tripadvisor.com.ph//Attraction_Review-g294208-d325044-Reviews-Lamu_Old_Town-Lamu_Island_Coast_Province.html',
 'https://www.tripadvisor.com.ph//Attraction_Review-g661279-d2281070-Reviews-Local_Ocean_Conservation-Watamu_Coast_Province.html',
 'https://www.tripadvisor.com.ph//Attraction_Review-g294207-d3334746-Reviews-African_Heritage_House-Nairobi.html',
 'https://www.tripadvisor.com.ph//Attraction_Review-g294207-d2345731-Reviews-Brown_s

In [11]:
final_first_page_list = []

In [12]:
for i in range(len(first_page_list)):
    final_first_page_list.append(first_page_list[i][0:30] + first_page_list[i][31:])

In [13]:
final_first_page_list

['https://www.tripadvisor.com.ph/Attraction_Review-g480204-d1643758-Reviews-Kiambethu_Farm-Limuru_Central_Province.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g1600222-d1924858-Reviews-Observation_Hill-Amboseli_National_Park_Amboseli_Eco_system_Rift_Valley_Province.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g294207-d1607150-Reviews-Karen_Blixen_Coffee_Garden_and_Cottages-Nairobi.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g661279-d1131514-Reviews-Bio_Ken_Snake_Farm-Watamu_Coast_Province.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g294208-d325044-Reviews-Lamu_Old_Town-Lamu_Island_Coast_Province.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g661279-d2281070-Reviews-Local_Ocean_Conservation-Watamu_Coast_Province.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g294207-d3334746-Reviews-African_Heritage_House-Nairobi.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g294207-d2345731-Reviews-Brown_s_Cheese_

**Getting All Review Pages from Initial Pages** 

In [14]:
final_url_list = []
skipped_urls = []
# url_list
# test1 = url_list[-2:] # less than 10 reviews test (1 page)
# test1

In [15]:
def getFinalList(x):
    for i in x:
        page_nos = []
        print('site: ', i)
        page = requests.get(i, headers = headers)
        soup = BeautifulSoup(page.content, 'html.parser')
        readcheck = soup.find_all(class_ = 'count')
        print('readcheck: ', len(readcheck))
        for j in soup.find_all('a'):
            if j.get('data-page-number') != None and readcheck != 0:
                page_nos.append(j.get('data-page-number'))
            elif j.get('data-page-number') == None and readcheck != 0:
                page_nos.append(0)
        if len(readcheck) == 0:
            page_nos = []
        page_nos = list(map(int, page_nos))
        if len(page_nos) > 0:
            final_url_list.append(i)
            if max(page_nos) > 0:
                getReviewPages(i, max(page_nos))
        if len(page_nos) == 0 and i not in skipped_urls:
            skipped_urls.append(i)
        if len(page_nos) > 0 and i in skipped_urls:
            skipped_urls.remove(i)
        print('skipped: ', skipped_urls)
        time.sleep(random.uniform(5, 15))
    if len(skipped_urls) != 0:
        print('restarting function...')
        getFinalList(skipped_urls)
        
def getReviewPages(x, y):
    pre = re.findall('https://www.tripadvisor.com.ph/Attraction_Review-\w\d+-\w\d+-Reviews-' , x)
    post = re.findall('Reviews-\D+' , x)
    print(len(pre))
    print(len(post))
    review_pages = ((pre[0] + 'or{}' + post[0].replace('Reviews', '')).format(j) for j in range(10, y*10, 10))
    final_url_list.extend(review_pages)

In [16]:
getFinalList(final_first_page_list)

site:  https://www.tripadvisor.com.ph/Attraction_Review-g480204-d1643758-Reviews-Kiambethu_Farm-Limuru_Central_Province.html
readcheck:  13
1
1
skipped:  []
site:  https://www.tripadvisor.com.ph/Attraction_Review-g1600222-d1924858-Reviews-Observation_Hill-Amboseli_National_Park_Amboseli_Eco_system_Rift_Valley_Province.html
readcheck:  21
1
1
skipped:  []
site:  https://www.tripadvisor.com.ph/Attraction_Review-g294207-d1607150-Reviews-Karen_Blixen_Coffee_Garden_and_Cottages-Nairobi.html
readcheck:  19
1
1
skipped:  []
site:  https://www.tripadvisor.com.ph/Attraction_Review-g661279-d1131514-Reviews-Bio_Ken_Snake_Farm-Watamu_Coast_Province.html
readcheck:  11
1
1
skipped:  []
site:  https://www.tripadvisor.com.ph/Attraction_Review-g294208-d325044-Reviews-Lamu_Old_Town-Lamu_Island_Coast_Province.html
readcheck:  16
1
1
skipped:  []
site:  https://www.tripadvisor.com.ph/Attraction_Review-g661279-d2281070-Reviews-Local_Ocean_Conservation-Watamu_Coast_Province.html
readcheck:  11
1
1
skipped:

site:  https://www.tripadvisor.com.ph/Attraction_Review-g303978-d3384386-Reviews-Mzima_Springs-Tsavo_National_Park_West_Coast_Province.html
readcheck:  17
1
1
skipped:  []
site:  https://www.tripadvisor.com.ph/Attraction_Review-g303975-d555773-Reviews-Lake_Nakuru-Lake_Nakuru_National_Park_Rift_Valley_Province.html
readcheck:  27
1
1
skipped:  []
site:  https://www.tripadvisor.com.ph/Attraction_Review-g2697619-d472318-Reviews-Lake_Naivasha-Rift_Valley_Province.html
readcheck:  21
1
1
skipped:  []
site:  https://www.tripadvisor.com.ph/Attraction_Review-g294207-d649051-Reviews-Ngong_Hills-Nairobi.html
readcheck:  16
1
1
skipped:  []
site:  https://www.tripadvisor.com.ph/Attraction_Review-g10298591-d472319-Reviews-Mount_Longonot_National_Park-Maai_Mahiu_Rift_Valley_Province.html
readcheck:  12
1
1
skipped:  []
site:  https://www.tripadvisor.com.ph/Attraction_Review-g661279-d2281070-Reviews-Local_Ocean_Conservation-Watamu_Coast_Province.html
readcheck:  11
1
1
skipped:  []
site:  https://ww

In [17]:
final_url_list

['https://www.tripadvisor.com.ph/Attraction_Review-g480204-d1643758-Reviews-Kiambethu_Farm-Limuru_Central_Province.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g480204-d1643758-Reviews-or10-Kiambethu_Farm-Limuru_Central_Province.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g480204-d1643758-Reviews-or20-Kiambethu_Farm-Limuru_Central_Province.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g480204-d1643758-Reviews-or30-Kiambethu_Farm-Limuru_Central_Province.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g480204-d1643758-Reviews-or40-Kiambethu_Farm-Limuru_Central_Province.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g480204-d1643758-Reviews-or50-Kiambethu_Farm-Limuru_Central_Province.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g480204-d1643758-Reviews-or60-Kiambethu_Farm-Limuru_Central_Province.html',
 'https://www.tripadvisor.com.ph/Attraction_Review-g480204-d1643758-Reviews-or70-Kiambethu_Farm-Limuru_Central_Provi

In [18]:
len(final_url_list)

3188

In [ ]:
final_url_list_set = list(set(final_url_list))


In [91]:
import pandas as pd
df_url = pd.DataFrame({'URL': final_url_list})
df_url.head()

,URL
0,https://www.tripadvisor.com.ph/Attraction_Revi...
1,https://www.tripadvisor.com.ph/Attraction_Revi...
2,https://www.tripadvisor.com.ph/Attraction_Revi...
3,https://www.tripadvisor.com.ph/Attraction_Revi...
4,https://www.tripadvisor.com.ph/Attraction_Revi...


In [92]:
df_url.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028 entries, 0 to 1027
Data columns (total 1 columns):
URL    1028 non-null object
dtypes: object(1)
memory usage: 8.2+ KB


In [93]:
df_url.to_csv('urls1.csv', index = False)

11/09/2019

In [1]:
import pandas as pd
df_url = pd.read_csv('urls1.csv')
df_url.head()

,URL
0,https://www.tripadvisor.com.ph/Attraction_Revi...
1,https://www.tripadvisor.com.ph/Attraction_Revi...
2,https://www.tripadvisor.com.ph/Attraction_Revi...
3,https://www.tripadvisor.com.ph/Attraction_Revi...
4,https://www.tripadvisor.com.ph/Attraction_Revi...


In [2]:
df_url.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028 entries, 0 to 1027
Data columns (total 1 columns):
URL    1028 non-null object
dtypes: object(1)
memory usage: 8.2+ KB


**Getting Wanted Data From Review Pages - Review Data, Customer Data, Destination Data, etc. (Unfinished)**

In [146]:
rev_place_list = []
rev_head_list = []
rev_body_list = []
rev_rating_list = []
rev_date_list = []
page_nos = []
skipped_urls = []

In [147]:
def getPlace(x, y):
    for i in y.find_all(class_ = 'noQuotes'):
        rev_place_list.append(x.find(id = 'HEADING').string)
        
def getHead(x):
    for i in x.find_all(class_ = 'noQuotes'):
        rev_head_list.append(i.string)

def getBody(x):
    for i in range(x.find_all(class_ = 'partial_entry'):
        rev_body_list.append(i.string)

def getRating(x):
    rev_revs = x.find_all(class_='ui_column is-9')
    for i in range(len(rev_revs)):
        if rev_revs[i].find(class_ = 'ui_bubble_rating bubble_50') != None:
            rev_rating_list.append(5)
        elif rev_revs[i].find(class_ = 'ui_bubble_rating bubble_40') != None:
            rev_rating_list.append(4)
        elif rev_revs[i].find(class_ = 'ui_bubble_rating bubble_30') != None:
            rev_rating_list.append(3)
        elif rev_revs[i].find(class_ = 'ui_bubble_rating bubble_20') != None:
            rev_rating_list.append(2)
        elif rev_revs[i].find(class_ = 'ui_bubble_rating bubble_10') != None:
            rev_rating_list.append(1)

def getDate(x):
    for i in x.find_all(class_ = 'prw_rup prw_reviews_stay_date_hsx'):
        rev_date_list.append(str(i.span.next_sibling).strip())

def delay(x, y):
    time.sleep(random.uniform(x, y))
    return None

def getData(urls):
    start_revs = 0
    end_revs = 0
    for i in urls:
        start_revs = len(rev_head_list)
        print('start: ', len(rev_head_list))
        page = requests.get(i, headers = headers)
        soup = BeautifulSoup(page.content, 'html.parser')
        rev = soup.find(id = 'REVIEWS')
        getPlace(soup, rev)
        getHead(rev)
        getBody(rev)
        getRating(rev)
        getDate(rev)
        delay(15, 30)
        end_revs = len(rev_head_list)
        print('end: ', len(rev_head_list))
        if end_revs == start_revs and i not in skipped_urls:
            skipped_urls.append(i)
        if end_revs > start_revs and i in skipped_urls:
            skipped_urls.remove(i)
        print(skipped_urls)
    if len(skipped_urls) != 0:
        print('restarting function...')
        getData(skipped_urls)

In [148]:
getData(final_url_list) # this takes forever

start:  0
end:  10
[]
start:  10
end:  10
['https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or10-Church_of_St_George-Lalibela_Amhara_Region.html']
start:  10
end:  20
['https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or10-Church_of_St_George-Lalibela_Amhara_Region.html']
start:  20
end:  30
['https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or10-Church_of_St_George-Lalibela_Amhara_Region.html']
start:  30
end:  30
['https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or10-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or40-Church_of_St_George-Lalibela_Amhara_Region.html']
start:  30
end:  30
['https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or10-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or40-Church

end:  144
['https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or10-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or40-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or50-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or80-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or130-Church_of_St_George-Lalibela_Amhara_Region.html']
start:  144
end:  144
['https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or10-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or40-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Revie

end:  184
['https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or10-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or40-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or50-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or80-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or130-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d5427774-Reviews-or10-Yemrehanna_Kristos-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d5427774-Reviews-or20-Yemrehanna_Kristos-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d5427774-Reviews-

end:  228
['https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or10-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or40-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or50-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or80-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or130-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d5427774-Reviews-or10-Yemrehanna_Kristos-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d5427774-Reviews-or20-Yemrehanna_Kristos-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d5427774-Reviews-

end:  268
['https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or10-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or40-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or50-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or80-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-or130-Church_of_St_George-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d5427774-Reviews-or10-Yemrehanna_Kristos-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d5427774-Reviews-or20-Yemrehanna_Kristos-Lalibela_Amhara_Region.html', 'https://www.tripadvisor.com.ph/Attraction_Review-g480193-d5427774-Reviews-

AttributeError: 'NoneType' object has no attribute 'next_sibling'

In [149]:
print(rev_place_list)
print(rev_head_list)
print(rev_body_list)
print(rev_date_list)
print(rev_rating_list)

['Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. George', 'Church of St. 

In [150]:
print(len(rev_place_list))
print(len(rev_head_list))
print(len(rev_body_list))
print(len(rev_rating_list))
print(len(rev_date_list))

288
288
288
288
284


**Scratch + Trying Pandas**

In [152]:
import pandas as pd
df1 = pd.DataFrame({'Destination Name': rev_place_list, 'Review Head': rev_head_list, 'Review Body': rev_body_list, 'Rating': rev_rating_list})
df1.head()

,Destination Name,Review Head,Review Body,Rating
0,Church of St. George,This site rival Petra and Machu Pichu come now...,This site is amazing. Dark passages and bridge...,5
1,Church of St. George,Stone Church,Definitely one of the sights to see while here...,5
2,Church of St. George,Incredible workmanship,None,5
3,Church of St. George,Fabulous Workmanship!,This is the church we see in all of the photos...,5
4,Church of St. George,Iconic church,"St George is the church everyone recognizes, w...",4


In [153]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 4 columns):
Destination Name    288 non-null object
Review Head         288 non-null object
Review Body         129 non-null object
Rating              288 non-null int64
dtypes: int64(1), object(3)
memory usage: 9.1+ KB


In [154]:
df1.to_csv('ethiopiadata1.csv', index = False)

In [155]:
df1.head()

,Destination Name,Review Head,Review Body,Rating
0,Church of St. George,This site rival Petra and Machu Pichu come now...,This site is amazing. Dark passages and bridge...,5
1,Church of St. George,Stone Church,Definitely one of the sights to see while here...,5
2,Church of St. George,Incredible workmanship,None,5
3,Church of St. George,Fabulous Workmanship!,This is the church we see in all of the photos...,5
4,Church of St. George,Iconic church,"St George is the church everyone recognizes, w...",4


In [157]:
df2 = pd.read_csv('ethiopiadata1.csv')
df2.head()

,Destination Name,Review Head,Review Body,Rating
0,Church of St. George,This site rival Petra and Machu Pichu come now...,This site is amazing. Dark passages and bridge...,5
1,Church of St. George,Stone Church,Definitely one of the sights to see while here...,5
2,Church of St. George,Incredible workmanship,NaN,5
3,Church of St. George,Fabulous Workmanship!,This is the church we see in all of the photos...,5
4,Church of St. George,Iconic church,"St George is the church everyone recognizes, w...",4


In [158]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 4 columns):
Destination Name    288 non-null object
Review Head         288 non-null object
Review Body         129 non-null object
Rating              288 non-null int64
dtypes: int64(1), object(3)
memory usage: 9.1+ KB


In [159]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 4 columns):
Destination Name    288 non-null object
Review Head         288 non-null object
Review Body         129 non-null object
Rating              288 non-null int64
dtypes: int64(1), object(3)
memory usage: 9.1+ KB


In [160]:
df3 = df2.dropna()
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 0 to 287
Data columns (total 4 columns):
Destination Name    129 non-null object
Review Head         129 non-null object
Review Body         129 non-null object
Rating              129 non-null int64
dtypes: int64(1), object(3)
memory usage: 5.0+ KB


In [162]:
df3.head()

,Destination Name,Review Head,Review Body,Rating
0,Church of St. George,This site rival Petra and Machu Pichu come now...,This site is amazing. Dark passages and bridge...,5
1,Church of St. George,Stone Church,Definitely one of the sights to see while here...,5
3,Church of St. George,Fabulous Workmanship!,This is the church we see in all of the photos...,5
4,Church of St. George,Iconic church,"St George is the church everyone recognizes, w...",4
5,Church of St. George,Iconic sight that doesn't disappoint.,"Church of St George, probably the most well kn...",5


In [164]:
df3['Rating'].value_counts()

5    107
4     20
3      2
Name: Rating, dtype: int64

In [172]:
# Major Problem/s:
# 1. Iterating through different kinds of sites
# 2. Getting Customer Data

# Minor Issue/s:
# 1. Review Body "More" Button
# 2. Date of Experience Missing